In [7]:
import requests
from bs4 import BeautifulSoup, NavigableString, Tag
import csv
from datetime import datetime
import pandas as pd
import time

In [19]:
# Raíz de la página sobre la que trabajaremos
url_root = "http://www.formula1.com"
# Página específica donde iniciaremos nuestra extracción de información
url_start = "https://www.formula1.com/en/results.html"

In [173]:
# Creamos un dataset vacio con la especificación del tipo de campo que vamos a ingerir
df1 = pd.DataFrame(columns=['year'], dtype=int)
df2 = pd.DataFrame(columns=['date'], dtype=str)
df3 = pd.DataFrame(columns=['country','circuit'], dtype=str)
df4 = pd.DataFrame(columns=['position','car_num'], dtype=int)
df5 = pd.DataFrame(columns=['name','surname','alias', 'team'], dtype=str)
df6 = pd.DataFrame(columns=['laps'], dtype=int)
df7 = pd.DataFrame(columns=['time'], dtype=str)
df8 = pd.DataFrame(columns=['points'], dtype=int)
Formula_1 = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], axis=1)

In [37]:
# Función extracción url relativos a los diferentes años de los que hay datos.

def F1_year_extract(link):
    # Realizamos la petición a la web de la Fórmula 1.
    connection = requests.get(link)

    # Comprobamos que la petición nos devuelve un Status Code = 200
    statusCode = connection.status_code

    # Si obtenemos los datos, realizamos el proceso, en caso contrario, terminamos.
    if statusCode == 200:
    
        # Creamos una lista vacía para alojar las url de cada año
        years_url_list = []
    
        # Descargamos la página raíz de los resultados
        content = requests.get(link).text
        soup = BeautifulSoup(content, "lxml")
        
        # Acotamos al código donde se especifican las url de cada año
        cod_url_years = soup.find('div', {'class': 'resultsarchive-filter-container'})
    
        # Acotamos a cada división que contiene la url que buscamos
        data_year = cod_url_years.find_all('li', {'class': 'resultsarchive-filter-item'})
    
        # Extraemos el año y la url donde buscaremos los resultados y lo incorporamos a la lista creada
        for dy in data_year:
            year = dy.find("span").getText()
            if year.isdigit():
                link = dy.find("a").get("href")
                url = "%s%s" % (url_root, link)
                years_url_list.append((year,url))  # Lista "path" años.
                
                
        return years_url_list
    else:
        print("Error de carga en la página inicial:",link)

In [144]:
# Función extracción url relativos a cada gran premio de cada año.

def F1_prix_extract(link):
    # Realizamos la petición a la web de la Fórmula 1.
    connection = requests.get(link)

    # Comprobamos que la petición nos devuelve un Status Code = 200
    statusCode = connection.status_code

    # Si obtenemos los datos, realizamos el proceso, en caso contrario, terminamos.
    if statusCode == 200:
        
        # Creamos una lista vacía para almacenar los link a cada evento concreto
        prix_url_list = []
        
        # Descargamos la página de cada año
        content = requests.get(link).text
        soup = BeautifulSoup(content, "lxml")
        
        
        # Seleccionamos la división de donde extreremos los links de los eventos.
        table = soup.find('table', {'class':'resultsarchive-table'})

        table_body = table.find('tbody')
        rows = table_body.find_all('tr')
        for row in rows:
            link = row.find('a', {'class':'dark bold ArchiveLink'}).get("href")
            country = row.find('a', {'href':link}).getText(strip=True)
            date = row.find('td',{'class':'dark hide-for-mobile'}).getText(strip=True)
            url = "%s%s" % (url_root, link)
            prix_url_list.append((country,date,url))
       
        return(prix_url_list)
    
    else:
        print("Error de carga en la página de un año concreto:",link)



In [163]:
# Función extracción información de resultados para cada gran premio.

def F1_data_extract(link):
        
    # Realizamos la petición a la web de la Fórmula 1.
    connection = requests.get(link)

    # Comprobamos que la petición nos devuelve un Status Code = 200
    statusCode = connection.status_code

    # Si obtenemos los datos, realizamos el proceso, en caso contrario, terminamos.
    if statusCode == 200:
        
        # Descargamos la página de cada evento
        content = requests.get(link).text
        soup = BeautifulSoup(content, "lxml")
        
        # Acotamos al código donde se especifica el nombre del circuito
        circuit = soup.find('span', {'class': 'circuit-info'}).getText()
        
        table = soup.find('table', {'class':'resultsarchive-table'})
        table_body = table.find('tbody')
        rows = table_body.find_all('tr')
        
        records = []
        
        for row in rows:
            cols = row.find_all('td')
            pos = cols[1].getText()
            no = cols[2].getText()
            name = cols[3].find('span',{'class':'hide-for-tablet'}).getText()
            surname = cols[3].find('span',{'class':'hide-for-mobile'}).getText()
            alias = cols[3].find('span',{'class':'uppercase hide-for-desktop'}).getText()
            team = cols[4].getText()
            laps = cols[5].getText()
            time = cols[6].getText()
            points = cols[7].getText()
            records.append((circuit,pos,no,name,surname,alias,team,laps,time,points))
        
        # Incorporamos un retraso de 1 segundo después de cara obtención de datos
        time.sleep(1)
        return(records)

    else:
        print("Error de carga en la página de un evento concreto:",link)


A partir de este punto está el programa que irá utilizando las funciones creadas para la recolección de informacion

In [39]:
# Obtenemos el listado de url de cada año.
F1_url_by_year = F1_year_extract(url_start)


In [145]:
# Obtenemos el listado de urls de cada gran premio según el año.
F1_event_url_list = []

# Vamos a extraer la información para cada url de cada año
for season in F1_url_by_year:
    
    # Identificamos la información que tenemos en cada elemento de la lista
    year = season[0]
    link = season[1]
    
    # Extraemos la lista de url de eventos por cada año
    url_event_list = F1_prix_extract(link)
    
    # Extraemos los datos de la lista para crear una única lista con la información que nos interesa.
    for event  in url_event_list:
        country = event[0]
        date = event[1]
        link_e = event[2]
        reg = (year,country,date,link_e)
        # Esta lista contendrá todas las url de todos los eventos.
        F1_event_url_list.append(reg)

In [177]:
# Obtenemos los resultados de cada carrera

#for race in F1_event_url_list:
for race in F1_event_url_list:

    # Identificamos la información que tenemos en cada elemento de la lista
    year = race[0]
    country = race[1]
    date = race[2]
    link = race[3]
    
    # Extraemos los datos de cada carrera
    data = F1_data_extract(link)
    
    #recorrer la lista data de los resultado de ese evento concreto
    for result in data:
        
        # Incorporar en el dataset cada registro.
        new_reg = {'year':year ,
                   'date':date,
                   'country':country, 
                   'circuit':result[0],
                   'position':result[1],
                   'car_num':result[2],
                   'name':result[3],
                   'surname':result[4],
                   'alias':result[5],
                   'team':result[6],
                   'laps':result[7],
                   'time':result[8],
                   'points':result[9]
                  }
        Formula_1.loc[len(Formula_1)] = new_reg
    

2020 Austria
2020 Austria
2020 Austria
2020 Austria
2020 Austria
2020 Austria
2020 Austria
2020 Austria
2020 Austria
2020 Austria
2020 Austria
2020 Austria
2020 Austria
2020 Austria
2020 Austria
2020 Austria
2020 Austria
2020 Austria
2020 Austria
2020 Austria
2020 Styria
2020 Styria
2020 Styria
2020 Styria
2020 Styria
2020 Styria
2020 Styria
2020 Styria
2020 Styria
2020 Styria
2020 Styria
2020 Styria
2020 Styria
2020 Styria
2020 Styria
2020 Styria
2020 Styria
2020 Styria
2020 Styria
2020 Styria
2020 Hungary
2020 Hungary
2020 Hungary
2020 Hungary
2020 Hungary
2020 Hungary
2020 Hungary
2020 Hungary
2020 Hungary
2020 Hungary
2020 Hungary
2020 Hungary
2020 Hungary
2020 Hungary
2020 Hungary
2020 Hungary
2020 Hungary
2020 Hungary
2020 Hungary
2020 Hungary
2020 Great Britain
2020 Great Britain
2020 Great Britain
2020 Great Britain
2020 Great Britain
2020 Great Britain
2020 Great Britain
2020 Great Britain
2020 Great Britain
2020 Great Britain
2020 Great Britain
2020 Great Britain
2020 Great B

2019 Abu Dhabi
2019 Abu Dhabi
2019 Abu Dhabi
2019 Abu Dhabi
2019 Abu Dhabi
2019 Abu Dhabi
2019 Abu Dhabi
2019 Abu Dhabi
2019 Abu Dhabi
2019 Abu Dhabi
2019 Abu Dhabi
2019 Abu Dhabi
2019 Abu Dhabi
2019 Abu Dhabi
2019 Abu Dhabi
2019 Abu Dhabi
2019 Abu Dhabi
2019 Abu Dhabi
2019 Abu Dhabi
2019 Abu Dhabi
2018 Australia
2018 Australia
2018 Australia
2018 Australia
2018 Australia
2018 Australia
2018 Australia
2018 Australia
2018 Australia
2018 Australia
2018 Australia
2018 Australia
2018 Australia
2018 Australia
2018 Australia
2018 Australia
2018 Australia
2018 Australia
2018 Australia
2018 Australia
2018 Bahrain
2018 Bahrain
2018 Bahrain
2018 Bahrain
2018 Bahrain
2018 Bahrain
2018 Bahrain
2018 Bahrain
2018 Bahrain
2018 Bahrain
2018 Bahrain
2018 Bahrain
2018 Bahrain
2018 Bahrain
2018 Bahrain
2018 Bahrain
2018 Bahrain
2018 Bahrain
2018 Bahrain
2018 Bahrain
2018 China
2018 China
2018 China
2018 China
2018 China
2018 China
2018 China
2018 China
2018 China
2018 China
2018 China
2018 China
2018 Chi

2017 Great Britain
2017 Great Britain
2017 Great Britain
2017 Great Britain
2017 Great Britain
2017 Great Britain
2017 Great Britain
2017 Great Britain
2017 Great Britain
2017 Great Britain
2017 Great Britain
2017 Great Britain
2017 Great Britain
2017 Great Britain
2017 Great Britain
2017 Great Britain
2017 Great Britain
2017 Great Britain
2017 Great Britain
2017 Great Britain
2017 Hungary
2017 Hungary
2017 Hungary
2017 Hungary
2017 Hungary
2017 Hungary
2017 Hungary
2017 Hungary
2017 Hungary
2017 Hungary
2017 Hungary
2017 Hungary
2017 Hungary
2017 Hungary
2017 Hungary
2017 Hungary
2017 Hungary
2017 Hungary
2017 Hungary
2017 Hungary
2017 Belgium
2017 Belgium
2017 Belgium
2017 Belgium
2017 Belgium
2017 Belgium
2017 Belgium
2017 Belgium
2017 Belgium
2017 Belgium
2017 Belgium
2017 Belgium
2017 Belgium
2017 Belgium
2017 Belgium
2017 Belgium
2017 Belgium
2017 Belgium
2017 Belgium
2017 Belgium
2017 Italy
2017 Italy
2017 Italy
2017 Italy
2017 Italy
2017 Italy
2017 Italy
2017 Italy
2017 Italy
2

2016 Mexico
2016 Mexico
2016 Mexico
2016 Mexico
2016 Mexico
2016 Mexico
2016 Mexico
2016 Mexico
2016 Mexico
2016 Mexico
2016 Mexico
2016 Mexico
2016 Mexico
2016 Mexico
2016 Mexico
2016 Mexico
2016 Mexico
2016 Mexico
2016 Mexico
2016 Mexico
2016 Mexico
2016 Mexico
2016 Brazil
2016 Brazil
2016 Brazil
2016 Brazil
2016 Brazil
2016 Brazil
2016 Brazil
2016 Brazil
2016 Brazil
2016 Brazil
2016 Brazil
2016 Brazil
2016 Brazil
2016 Brazil
2016 Brazil
2016 Brazil
2016 Brazil
2016 Brazil
2016 Brazil
2016 Brazil
2016 Brazil
2016 Brazil
2016 Abu Dhabi
2016 Abu Dhabi
2016 Abu Dhabi
2016 Abu Dhabi
2016 Abu Dhabi
2016 Abu Dhabi
2016 Abu Dhabi
2016 Abu Dhabi
2016 Abu Dhabi
2016 Abu Dhabi
2016 Abu Dhabi
2016 Abu Dhabi
2016 Abu Dhabi
2016 Abu Dhabi
2016 Abu Dhabi
2016 Abu Dhabi
2016 Abu Dhabi
2016 Abu Dhabi
2016 Abu Dhabi
2016 Abu Dhabi
2016 Abu Dhabi
2016 Abu Dhabi
2015 Australia
2015 Australia
2015 Australia
2015 Australia
2015 Australia
2015 Australia
2015 Australia
2015 Australia
2015 Australia
2015 Au

2014 Germany
2014 Germany
2014 Germany
2014 Germany
2014 Germany
2014 Germany
2014 Germany
2014 Germany
2014 Germany
2014 Germany
2014 Germany
2014 Germany
2014 Germany
2014 Germany
2014 Germany
2014 Germany
2014 Germany
2014 Germany
2014 Germany
2014 Germany
2014 Germany
2014 Germany
2014 Hungary
2014 Hungary
2014 Hungary
2014 Hungary
2014 Hungary
2014 Hungary
2014 Hungary
2014 Hungary
2014 Hungary
2014 Hungary
2014 Hungary
2014 Hungary
2014 Hungary
2014 Hungary
2014 Hungary
2014 Hungary
2014 Hungary
2014 Hungary
2014 Hungary
2014 Hungary
2014 Hungary
2014 Hungary
2014 Belgium
2014 Belgium
2014 Belgium
2014 Belgium
2014 Belgium
2014 Belgium
2014 Belgium
2014 Belgium
2014 Belgium
2014 Belgium
2014 Belgium
2014 Belgium
2014 Belgium
2014 Belgium
2014 Belgium
2014 Belgium
2014 Belgium
2014 Belgium
2014 Belgium
2014 Belgium
2014 Belgium
2014 Belgium
2014 Italy
2014 Italy
2014 Italy
2014 Italy
2014 Italy
2014 Italy
2014 Italy
2014 Italy
2014 Italy
2014 Italy
2014 Italy
2014 Italy
2014 Italy

...

1960 Monaco
1960 Monaco
1960 Monaco
1960 Monaco
1960 Monaco
1960 Monaco
1960 Monaco
1960 Monaco
1960 Monaco
1960 Monaco
1960 Monaco
1960 Monaco
1960 Monaco
1960 Monaco
1960 Monaco
1960 Monaco
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Indianapolis 500
1960 Netherlands
1960 Netherlands
1960 Netherlands
1960 Netherlands
1960 Netherlan

1958 Germany
1958 Portugal
1958 Portugal
1958 Portugal
1958 Portugal
1958 Portugal
1958 Portugal
1958 Portugal
1958 Portugal
1958 Portugal
1958 Portugal
1958 Portugal
1958 Portugal
1958 Portugal
1958 Portugal
1958 Portugal
1958 Italy
1958 Italy
1958 Italy
1958 Italy
1958 Italy
1958 Italy
1958 Italy
1958 Italy
1958 Italy
1958 Italy
1958 Italy
1958 Italy
1958 Italy
1958 Italy
1958 Italy
1958 Italy
1958 Italy
1958 Italy
1958 Italy
1958 Italy
1958 Italy
1958 Italy
1958 Morocco
1958 Morocco
1958 Morocco
1958 Morocco
1958 Morocco
1958 Morocco
1958 Morocco
1958 Morocco
1958 Morocco
1958 Morocco
1958 Morocco
1958 Morocco
1958 Morocco
1958 Morocco
1958 Morocco
1958 Morocco
1958 Morocco
1958 Morocco
1958 Morocco
1958 Morocco
1958 Morocco
1958 Morocco
1958 Morocco
1958 Morocco
1958 Morocco
1957 Argentina
1957 Argentina
1957 Argentina
1957 Argentina
1957 Argentina
1957 Argentina
1957 Argentina
1957 Argentina
1957 Argentina
1957 Argentina
1957 Argentina
1957 Argentina
1957 Argentina
1957 Argentina


1955 Belgium
1955 Belgium
1955 Belgium
1955 Belgium
1955 Belgium
1955 Belgium
1955 Belgium
1955 Belgium
1955 Belgium
1955 Belgium
1955 Belgium
1955 Belgium
1955 Belgium
1955 Belgium
1955 Netherlands
1955 Netherlands
1955 Netherlands
1955 Netherlands
1955 Netherlands
1955 Netherlands
1955 Netherlands
1955 Netherlands
1955 Netherlands
1955 Netherlands
1955 Netherlands
1955 Netherlands
1955 Netherlands
1955 Netherlands
1955 Netherlands
1955 Netherlands
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Great Britain
1955 Italy
1955 Italy
1955 Italy


1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1953 Italy
1952 Switzerland
1952 Switzerland
1952 Switzerland
1952 Switzerland
1952 Switzerland
1952 Switzerland
1952 Switzerland
1952 Switzerland
1952 Switzerland
1952 Switzerland
1952 Switzerland
1952 Switzerland
1952 Switzerland
1952 Switzerland
1952 Switzerland
1952 Switzerland
1952 Switzerland
1952 Switzerland
1952 Switzerland
1952 Switzerland
1952 Switzerland
1952 Switzerland
1952 Indianapolis 500
1952 Indianapolis 500
1952 Indianapolis 500
1952 Indianapolis 500
1952 Indianapolis 500
1952 Indianapolis 500
1952 Indianapolis 500
1952 Indianapolis 500
1952 Indianapolis 500
1952 Indianapolis 500
1952 Indianapolis 500
1952 Indianapolis 500
1952 Indianapolis 500

1950 Belgium
1950 Belgium
1950 Belgium
1950 Belgium
1950 Belgium
1950 Belgium
1950 Belgium
1950 Belgium
1950 Belgium
1950 Belgium
1950 Belgium
1950 Belgium
1950 Belgium
1950 Belgium
1950 France
1950 France
1950 France
1950 France
1950 France
1950 France
1950 France
1950 France
1950 France
1950 France
1950 France
1950 France
1950 France
1950 France
1950 France
1950 France
1950 France
1950 France
1950 France
1950 France
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy
1950 Italy


In [178]:
# Escribimos el dataset obtenido en un fichero csv
Formula_1.to_csv('Formula_1_results.csv', index=False)